# Geographical analysis for hydrological modelling

This notebook shows how to delineate a catchment and extract properties from a digital elevation model (DEM) and a land-use data set. The processes are hosted on the Raven server, which in the background connects to a GeoServer instance to query watershed contours, DEM and land-use data. 

We connect to Raven's Web Processing Service interface using birdy's `WPSClient`. 

In [ ]:
# Cookie-cutter template required to connect to the PAVICS-Hydro Raven WPS server

from birdy import WPSClient
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import os

# Set environment variable RAVEN_WPS_URL to "http://localhost:9099" to run on the default local server
# url = os.environ.get("RAVEN_WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps")
url = os.environ.get("RAVEN_WPS_URL", "http://localhost:9099")

wps = WPSClient(url)

We first extract the watershed contour for the point of interest. The process looks into the HydroSheds databast to finds the watershed enclosing the given location. The `location` parameter identifies the outlet of the watershed, and `aggregate_upstream` determines whether or not we want the service to return the union of all upstream basins. Here we set it to `False` to reduce the size of the basin and speed-up computations. 

The output of the `hydrosheds-select` process is a GeoJSON geometry. 

In [ ]:
r_select = wps.hydrobasins_select(location="-71.291660, 50.492758", aggregate_upstream=False)

In [ ]:
# Get GeoJSON polygon of the delineated catchment.
# We can either get links to the files stored on the server, or get the data directly. 
[feature_url, upstream_basins_url] = r_select.get(asobj=False)
[feature, upstream_basins] = r_select.get(asobj=True)

We can now plot the outline of the watershed by loading it into `GeoPandas`. 

In [ ]:
# df = gpd.read_file(feature_url)
df = gpd.GeoDataFrame.from_features([feature])
df.plot()

Now that we have delineated a catchment, lets find the zonal statistics and other properties of the catchment. We can pass the catchment outline either as a link to a file (e.g. `feature_url`), or pass the data directly as a string. 

In [ ]:
# Here we are using the geojson file created on the server in the last step as the input value to a process 
# computing watershed properties. 
#crs=4326
#projected_crs=32198
resp = wps.shape_properties(shape=json.dumps(feature))

Now, let's extract the data from the WPS service response:

In [ ]:
[properties, ]=resp.get(asobj=True)
prop = properties[0]
print(prop)

area = prop['area']/1000000.0
longitude = prop['centroid'][0]
latitude = prop['centroid'][1]
gravelius = prop['gravelius']
perimeter = prop['perimeter']

shapeProperties = {'area':area, 'longitude':longitude, 'latitude':latitude, 'gravelius':gravelius, 'perimeter':perimeter}
shapeProperties

Note that these properties are a mix of the properties of the original file where the shape is stored, and properties computed by the process (area, centroid, perimeter and gravelius). Note also that the computed area is in m², while the "SUB_AREA" property is in km², and that there are slight differences between the two values. 

Now we'll extract the land-use properties of the watershed. We pass the link to the watershed outline to a process using in the background the [North American Land Change Monitoring System](http://www.cec.org/tools-and-resources/north-american-environmental-atlas/north-american-land-change-monitoring-system) dataset, and retrieve properties over the region.  

In [ ]:
# Use the geoserver to extract the land cover over the appropriate bounding box (automatic)
resp = wps.nalcms_zonal_stats(shape=feature_url, select_all_touching=True, band=1, simple_categories=True)

In [ ]:
# Note that geojson needs to be installed for this to work. 
# $ pip install -r requirements_extra.txt
features, statistics  = resp.get(asobj=True)
print(features[0]['properties'], '\n\n')
print(statistics)

We now have the statistics from the NALCMS zonal_stats toolbox regarding the land use, from which we calculate the ratio of each land-use component.

In [ ]:
# total = sum(lu.values())
lu = statistics[0]
total = sum(lu.values())

landUse = {k: (v / total) for (k,v) in lu.items()}
landUse

The next step will be to collect terrain data, such as elevation, slope and aspect. We will do this using the `terrain_analysis` WPS service:

In [ ]:
resp = wps.terrain_analysis(shape=feature_url, select_all_touching=True, projected_crs=3978)

Now let's extract the properties from the WPS response. Use `asobj=True` to have Birdy preprocess the data and return the data directly.

In [ ]:
properties, dem = resp.get(asobj=True)

elevation=properties[0]['elevation']
slope=properties[0]['slope']
aspect=properties[0]['aspect']

terrain_data={'elevation':elevation, 'slope':slope,'aspect':aspect}

Finally, display all the extracted parameters for the user:

In [ ]:
all_properties={**shapeProperties, **landUse, **terrain_data}
print(all_properties)

Note here that while the feature outline is defined above in terms of geographic coordinates (latitude, longitude), the DEM is projected onto a 2D cartesion coordinate system (here NAD83, the Canada Atlas Lambert projection). This is necessary to perform slope calculations. 

For more information on this, see: https://en.wikipedia.org/wiki/Map_projection

In [ ]:
# NBVAL_SKIP
import cartopy.crs as ccrs
import rasterio
from rasterio.plot import show
import xarray as xr
import rasterio
from rasterio.io import MemoryFile

with MemoryFile(dem) as mem:
    with mem.open(driver='gtiff') as src:
        crs = ccrs.LambertConformal(central_latitude=49, central_longitude=-95, standard_parallels=(49, 77))
        da = xr.open_rasterio(src)
        da.name = 'Elevation'
        da.attrs['units'] = 'm'
        ax = plt.subplot(projection=crs)
        da.where(da!=-32768).sel(band=1).plot.imshow(ax=ax, transform=crs)
        plt.show()